In [1]:
!pip install feedparser
!pip install newspaper3k        # 파이썬 3버전
!pip install konlpy             # 한국어 문법 분석위해서 설치

     |████████████████████████████████| 194kB 6.1MB/s 
  Created wheel for feedparser: filename=feedparser-5.2.1-cp36-none-any.whl size=44940 sha256=ee153c6c3d81722ee60e5b234c0b1de15c9b1d91b28a387325d9891b0c00a609
  Stored in directory: /root/.cache/pip/wheels/8c/69/b7/f52763c41c5471df57703a0ef718a32a5e81ee35dcf6d4f97f
Successfully built feedparser
     |████████████████████████████████| 215kB 6.5MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 7.4MB 9.8MB/s 
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp36-none-any.whl size=3357 sha256=262c97defa915891d4983bd410a367ac81d8ae39566b6f0b4567e678a351abda
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp36-none-any.whl size=13539 sha256=7bc9618de5a1c7c0f92513961989ab0751521f2d793d1130bb85761b6c58f626
  Stored in directory: /root/.cache/pip/wheels/81/2b/43

In [2]:
import feedparser
from newspaper import Article
from konlpy.tag import Okt
from collections import Counter
from operator import eq
import numpy as np

urls = ("http://rss.etnews.com/Section901.xml",
         "http://rss.etnews.com/Section902.xml",
         "http://rss.etnews.com/Section903.xml", 
        None)

def crawl_rss(urls):
  arr_rss=[]
  for url in urls:
    parse_rss = feedparser.parse(url)
    for p in parse_rss.entries:
      arr_rss.append({'title':p.title, 'link':p.link})
  return arr_rss


def crawl_article(url, language = 'ko'):
  # print(["Crawl Article"], url)
  var_article = Article(url, language = language)   #해당 url의 기사를 가져옴
  var_article.download()
  var_article.parse()
  return var_article.title, var_article.text


def main():
  file = open('/content/stopwords.txt','r')
  line = file.readlines()
  file.close
  stopwords =  []
  for l in line:
    stopwords.append(l[:-1])                                # txt파일로부터 한국어 불용어 읽고, stopwords 라는 리스트에 저장
  print('== 한국어 불용어 리스트 ==')
  print(stopwords)
  print("")

  # 기사 가져옴
  spliter = Okt()                  
  article_list = crawl_rss(urls)
  
  for article in article_list:
    _, text = crawl_article(article['link'])
    article['text'] = text
  
  key_words = {}
  for article in article_list:
    sp = Okt()
    noun = sp.nouns(article['text'])                        # 기사를 명사단위로 나누고 noun 리스트에 넣음
    for n in noun:
      if n in stopwords:                                    # noun 리스트 안에 불용어가 있으면, key_words에 넣지 않음
        continue
      else :
        if n in list(key_words.keys()):
          key_words[n]+=1                                   # 저장된 key_words 중에서 noun 항목과 일치하는 것이 있으면 개수를 1 증가
        else:
          key_words[n] = 1                                  # 저장된 key_words 중에서 noun 항목과 일치하는 것이 없으면 새롭게 추가
        
  i = 1
  count = Counter(key_words)
  for rank in count.most_common(10):
    print(i,'등  키워드 / 횟수 / 빈도')                     # 각 키워드가 전체 기사 속에서 등장한 횟수, 빈도를 상위 10개 출력
    print("     ",rank[0],'   ',rank[1],'  ','%.2f'%(rank[1]/len(key_words)*100),'%')
    print('')
    i = i+1


if __name__ == "__main__":
  main()

== 한국어 불용어 리스트 ==
['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수 없다', '해서는 안된다', '뿐만 아니라', '만이 아니다', '만은 아니다', '막론하고', '관계없이', '그치지 않다', '그러나', '그런데', '하지만', '든간에', '논하지 않다', '따지지 않다', '설사', '비록', '더라도', '아니면', '만 못하다', '하는 편이 낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이 외에', '이 밖에', '하여야', '비로소', '한다면 몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할 생각이다', '하려고하다', '이리하여', '그리하여', '그렇게 함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에 틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만', '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼', '남짓', '여', '얼마간', '약간', '다소', '좀', '조금', '다수', '몇', '얼마', '지만', '하물며', '또한', '그러나', '그렇지만', '하지만', '이외에도', '대해 말하자면', '뿐이다', '다음에', '반대로', '반대